In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# train.py

In [1]:
import sys
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import TimeDistributed, Bidirectional
from tensorflow.keras.layers import BatchNormalization

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

In [3]:
# # Check if script get enough parameters
# if len(sys.argv) < 6:
#         raise ValueError(
#            'Not enough paramters! \nUsage : python train.py MODEL_NAME EPOCHS DATA_DIR N_INPUT ENCODE (DIM)')
# ENCODED = sys.argv[5].lower() == 'true'

# if ENCODED:
#     if len(sys.argv) < 7:
#         raise ValueError(
#            'Not enough paramters! \nUsage : python train.py MODEL_NAME EPOCHS DATA_DIR N_INPUT ENCODE DIM')
#     else:    
#         N_OUTPUT = int(sys.argv[6])  # Representation dimensionality
# else:
#     N_OUTPUT = 192 * 2  # Number of Gesture Feature (position + velocity)

ENCODED = 1
N_OUTPUT = 180
EPOCHS = 70
DATA_DIR = 'data_3d_cord'
N_INPUT = 26  # Number of input features

BATCH_SIZE = 2056
N_HIDDEN = 256

N_CONTEXT = 60 + 1  # The number of frames in the context

In [5]:
def train(model_file):
    """
    Train a neural network to take speech as input and produce gesture as an output

    Args:
        model_file: file to store the model

    Returns:

    """

    # Get the data
    X = np.load(DATA_DIR + '/X_train.npy') #audio

    if ENCODED:
        print("check!")
        # If we learn speech-representation mapping we use encoded motion as output
        Y = np.load(DATA_DIR + '/' + str(N_OUTPUT)+ '/Y_train_encoded.npy') # motion
        # Correct the sizes
        train_size = min(X.shape[0], Y.shape[0])
        X = X[:train_size]
        Y = Y[:train_size]

    else:
        Y = np.load(DATA_DIR + '/Y_train.npy')

    N_train = int(len(X)*0.9)
    N_validation = len(X) - N_train

    # Split on training and validation
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=N_validation)

    # Define Keras model

    model = Sequential()
    model.add(TimeDistributed(Dense(N_HIDDEN), input_shape=(N_CONTEXT, N_INPUT)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(TimeDistributed(Dense(N_HIDDEN)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(TimeDistributed(Dense(N_HIDDEN)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))

    model.add(GRU(N_HIDDEN, return_sequences=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(N_OUTPUT))
    model.add(Activation('linear'))

    print(model.summary())

    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    hist = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(X_validation, Y_validation))
     
    model.save(model_file)

    # Save convergence results into an image
    pyplot.plot(hist.history['loss'], linewidth=3, label='train')
    pyplot.plot(hist.history['val_loss'], linewidth=3, label='valid')
    pyplot.grid()
    pyplot.legend()
    pyplot.xlabel('epoch')
    pyplot.ylabel('loss')
    pyplot.show()
    pyplot.savefig(model_file.replace('hdf5', 'png'))

In [6]:
# if __name__ == "__main__":

#     train(sys.argv[1])
train('first_model_3d_180_dim')

check!
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 61, 256)           6912      
_________________________________________________________________
batch_normalization (BatchNo (None, 61, 256)           1024      
_________________________________________________________________
activation (Activation)      (None, 61, 256)           0         
_________________________________________________________________
dropout (Dropout)            (None, 61, 256)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 61, 256)           65792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 256)           1024      
__________________________________

40435/40435 [==============================] - 4s 101us/sample - loss: 0.0079 - val_loss: 0.0067
Epoch 43/100
40435/40435 [==============================] - 4s 102us/sample - loss: 0.0077 - val_loss: 0.0067
Epoch 44/100
40435/40435 [==============================] - 4s 102us/sample - loss: 0.0075 - val_loss: 0.0061 - lo
Epoch 45/100
40435/40435 [==============================] - 4s 103us/sample - loss: 0.0074 - val_loss: 0.0061s - los
Epoch 46/100
40435/40435 [==============================] - 4s 106us/sample - loss: 0.0073 - val_loss: 0.0065
Epoch 47/100
40435/40435 [==============================] - 4s 101us/sample - loss: 0.0072 - val_loss: 0.0067
Epoch 48/100
40435/40435 [==============================] - 4s 102us/sample - loss: 0.0070 - val_loss: 0.0069
Epoch 49/100
40435/40435 [==============================] - 4s 102us/sample - loss: 0.0068 - val_loss: 0.0070
Epoch 50/100
40435/40435 [==============================] - 4s 102us/sample - loss: 0.0068 - val_loss: 0.0065: 2s -
Epoch

C:\Users\Wen\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:75: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [ ]:
Y = np.load(DATA_DIR + '/' + str(N_OUTPUT)+ '/Y_train_encoded.npy') 
Y

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('first_model_8_dim')

AttributeError: 'Sequential' object has no attribute 'history'

In [8]:
pyplot.plot(hist.history['loss'], linewidth=3, label='train')
pyplot.plot(hist.history['val_loss'], linewidth=3, label='valid')
pyplot.grid()
pyplot.legend()
pyplot.xlabel('epoch')
pyplot.ylabel('loss')

AttributeError: 'Sequential' object has no attribute 'history'